### Import Libraries

In [ ]:
import os
import pathlib
import numpy as np

from PIL import Image
from glob import glob

import tensorflow as tf
import matplotlib.pyplot as plt

### Get image files

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

data_dir

### View image by reading

In [ ]:
daisy_path = f'{data_dir}/daisy'
daisy_file = os.listdir(daisy_path)

daisy_file[:2]

In [ ]:
for img_file in daisy_file[:2]:
    img = Image.open(os.path.join(daisy_path, img_file)).resize((224,224))
    plt.title(img_file + ' : Positive')
    plt.imshow(img)
    plt.show()

### Data Preprocess

### We can make image-label dataset all-in-once by using image_dataset_from_directory

In [ ]:
# Image path
img_path = data_dir

# Hyperparameters
input_shape = (224, 224, 3)
batch_size = 32
num_classes = 5
seed = 42
valid_rate = 0.2

# Train Dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=img_path,
    label_mode="categorical", # binary / categorical
    batch_size=batch_size,
    image_size=(224, 224),    # input shape
    seed=seed,
    shuffle=True,
    validation_split=valid_rate,
    subset="training",           
)

# Test Dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=img_path,
    label_mode="categorical", # binary / categorical
    batch_size=batch_size,
    image_size=(224, 224),    # input shape
    seed=seed,
    validation_split=valid_rate,
    subset="validation", 
)

train_ds.class_names

In [ ]:
# 40,000건 중에서 32,000건 Train 사용. test용으로 8,000건 사용
len(train_ds) * 32 , len(test_ds) * 32

In [ ]:
batch_img, batch_label = next(iter(train_ds))
batch_img.shape, batch_label.shape

In [ ]:
# See sample image
for idx, (batch_img, batch_label) in enumerate(train_ds.take(1)):
  print(batch_img[idx].shape)
  print(batch_label[idx])
  plt.imshow(batch_img[idx] / 255.0)
  break

### Build Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Rescaling

# Hyperparameter Tunning
num_epochs = 10
learning_rate = 0.001
dropout_rate = 0.5
input_shape = (224, 224, 3)
num_classes = 5

model = Sequential()
model.add(Rescaling(1.0 / 255.0))  # without rescaling, the model's output will be poor
model.add(Conv2D(32, (5,5), (1,1), padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(64, (2,2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
    loss='categorical_crossentropy',  # Loss Function
    metrics=['accuracy']  # Metrics / Accuracy
)

### Callback

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

checkpoint_path = "my_checkpoint_2.keras"
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,
    monitor='val_loss',
    verbose=1,
)

### Training Model

In [ ]:
history = model.fit(
    train_ds,
    validation_data=(test_ds),
    epochs=num_epochs,
    callbacks=[es, checkpoint]
)

### Performance Graph

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.show()

### Prediction

In [ ]:
batch_img , batch_label = next(iter(test_ds))
type(batch_img), batch_img.shape

In [ ]:
# Predict model's accuracy by using test dataset
plt.figure(figsize=(16, 30))
for idx, (img, label) in enumerate(list(zip(batch_img, batch_label))):
    pred = model.predict(img.numpy().reshape(-1, 224, 224, 3), verbose=0)
    pred_t = np.argmax(pred)
    plt.subplot(8, 4, idx+1)
    plt.title(f'True Value:{np.argmax(label)}, Pred Value: {pred_t}')
    plt.imshow(img / 255.0)  # the value of image pixels is in range 0 to 254 so we should re-scale by 255
